# Outline of Framework
- Create Database Connection
- Create Table
- Exploration Data

## Create Database Connection

In [ ]:
import pandas as pd
import glob
import re
import io
from sqlalchemy.engine import url as sa_url
from sqlalchemy import create_engine
from connection import connectionDB

db_connect_url = sa_url.URL(
            drivername='postgresql+psycopg2',
            username=connectionDB.config.get('username'),
            password=connectionDB.config.get('password'),
            host=connectionDB.config.get('host'),
            port=str(connectionDB.config.get('port')),
            database=connectionDB.config.get('database'),
            
)

engine= create_engine(db_connect_url)

## Create Table 
### Create a function for initiate a list of file CSV and dbname

In [ ]:
def insert_into (num):
    pathdir= r'data'
    allFiles = glob.glob(pathdir + "/*.csv")
    while True:
        try:
            r1 = str(re.findall(r"/+[a-zA-Z]*",allFiles[num]))
            tabname = re.findall(r"[a-zA-Z]+",r1)
            r3 = re.findall(r"\w+/\w+.\w+",allFiles[num])
        except ValueError:
            print("Please enter a valid range number of index for table")
        else:
            break
    return r3, tabname

### Reading multiple file csv from list

In [ ]:
def multi_name_csv (num):
    csv_list=[]
    for j in range(num):
        csv_list.append(insert_into(j))
        df = pd.DataFrame(csv_list, columns =['csv_name', 'dbname'])
        csv_name = df.iloc[:, [0]]
        dname = df.iloc[:, [1]]
    return csv_name, dname

def read_multi_csv(num_list_csv,num_index):
    list_csv= multi_name_csv(num_list_csv)
    df_list = []
    for i in range(num_index):
        df_list.append(pd.read_csv('{x}'.format(x=list_csv[0].iloc[i][0][0]), encoding= 'unicode_escape', dtype='unicode'))
    return df_list

Check and Clean each result file dataframe from CSV input, to check each file, you can access it based on index, like this:

- check data + name column
    - df['number of index'].head()
- show list name column
    - list(df['number of index'].columns)
- rename column
    - df['number of index']['number of index']= df['number of index'].rename(columns = {'name column':'new name column'})
- drop column by index
    - df['number of index'].drop(df['number of index'].columns['number of index'], axis=1, inplace=True)
- etc

### Ingest dataframe into db

In [ ]:
def ingest_df_To_DB (num, num_a, num_b, num_c):
    de=read_csv(num,num_a)
    namedb= multi_name_csv(num_b)
    for i in range(num_c):
        de[i].to_sql('{x}'.format(x=namedb[1].iloc[i][0][0]), con=engine, if_exists = 'replace', index=False)

def main():
    ingest_df_To_DB(12,12,2,2)


if __name__ == "__main__":
    main()